#Cheatcode

## Colar no console JS como medida paliativa para desconexão por inatividade

```
function ConnectButton(){
    console.log("Connect pushed"); 
    document.querySelector("#top-toolbar > colab-connect-button").shadowRoot.querySelector("#connect").click() 
}

var colab = setInterval(ConnectButton,60000);
```



#Conexão com o drive para ter acesso as simulações.

In [ ]:
try:
  from google.colab import drive
  drive.mount('/content/drive', force_remount=True)
  COLAB = True
  print("Note: Using Google Colab")
except:
  print("Note: not using Google Colab")
  COLAB = False

Mounted at /content/drive
Note: Using Google Colab



---
#Importação de bibliotecas

In [ ]:
import pandas
import numpy as np
import os
import keras

#Visualização
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

from sklearn.metrics import mean_squared_error, max_error, mean_absolute_error

#RNN's
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, GRU, Dropout
from tensorflow.keras.callbacks import ModelCheckpoint

#Separação de fases

In [ ]:
  #parâmetros
  taxa_amostragem = 10000
  frequencia_sistema = 60
  ciclos_pos_falta = 2
  numero_amostras_ciclo = round(taxa_amostragem / frequencia_sistema)
  dados_por_fase = numero_amostras_ciclo*ciclos_pos_falta

#Load data

In [ ]:
baseDir = '/content/drive/MyDrive/ColabNotebooks/DATA/localizacao/'

In [ ]:
X_train = np.fromfile(baseDir + 'X_train', dtype=float, count=-1, sep='', offset=0).reshape(100800, 3, dados_por_fase)
X_validation = np.fromfile(baseDir + 'X_validation', dtype=float, count=-1, sep='', offset=0).reshape(33600, 3, dados_por_fase)

In [ ]:
X_train_t = np.fromfile(baseDir + 'X_train_t', dtype=float, count=-1, sep='', offset=0).reshape(100800, 3, dados_por_fase)
X_validation_t = np.fromfile(baseDir + 'X_validation_t', dtype=float, count=-1, sep='', offset=0).reshape(33600, 3, dados_por_fase)

In [ ]:
y_train = np.fromfile(baseDir + 'y_train', dtype=float, count=-1, sep='', offset=0)
y_validation =  np.fromfile(baseDir + 'y_validation', dtype=float, count=-1, sep='', offset=0)

#Criação do modelo

##Gated Recurrent Unit

In [ ]:
def gru():
  model = Sequential()

  model.add(GRU(1000,
                return_sequences=True,
                input_shape=(X_train.shape[1]*2, dados_por_fase)
                ))
  
  model.add(GRU(1000,
                return_sequences=True))
  
  model.add(GRU(1000))

  model.add(Dense(1))
  model.compile(loss='mean_squared_error',
                optimizer='adam',
                metrics=['mean_squared_error'])

  return model

##Sumário

In [ ]:
if 'gruModel' in globals():
  del(gruModel)
gruModel = gru()
gruModel.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 6, 1000)           4008000   
                                                                 
 gru_1 (GRU)                 (None, 6, 1000)           6006000   
                                                                 
 gru_2 (GRU)                 (None, 1000)              6006000   
                                                                 
 dense (Dense)               (None, 1)                 1001      
                                                                 
Total params: 16,021,001
Trainable params: 16,021,001
Non-trainable params: 0
_________________________________________________________________


#Treinamento GRU

##Callbacks

In [ ]:
log_directory = '/content/drive/MyDrive/ColabNotebooks/Models'

checkpoint = ModelCheckpoint( log_directory + '/' + 'best_gru_model_corrente_bus_8_9.hdf5',
                             monitor='val_mean_squared_error',
                             verbose=1,
                             save_best_only=True,
                             mode='auto',
                             period=1)

##Alteração para o formato de entrada

In [ ]:
if 'X_train' in globals():
  X_train_aux = np.hstack((X_train, X_train_t))
  del(X_train, X_train_t)
  X_validation_aux = np.hstack((X_validation, X_validation_t))
  del(X_validation, X_validation_t)

##Treinamento

In [ ]:
gruModel.fit(X_train_aux,
             y_train,
             epochs=5000,
             batch_size=1680,
             verbose=1,
             callbacks=[checkpoint],
             validation_data=(X_validation_aux, y_validation)
             )

## Avaliando o Modelo

In [ ]:
losses = pandas.DataFrame(gruModel.history.history)

In [ ]:
losses[['loss','val_loss']].plot(figsize=(70,30))

In [ ]:
losses[['mean_squared_error','val_mean_squared_error']].plot(figsize=(70,30))